In [1]:
import pandas as pd # for storing data in pandas dataframe
import sqlite3 # for accessing SQL database
data = sqlite3.connect("data.db") # create/access database

# clean
url = "NBA_Player_Stats_2.csv"
stats = pd.read_csv(url)

stats = stats.drop({"PF", "Age", "FGA", "GS", "G", "DRB", "3PA", "2P", "2PA", "2P%", "eFG%", "FTA", "ORB", "DRB"}, axis = 1)

stats["Tm"] = stats["Tm"].replace({"VAN": "MEM", "SEA" : "OKC", "NJN" : "BRK", "CHO" : "CHH", "NOK" : "NOP", "NOH" : "NOP"})

stats = stats.rename(columns = {"FT%":"FT_P", "3P%":"THRP_P","3P":"THRP", "FG%":"FG_P"})

stats = stats[stats['Tm'] != "TOT"]

stats.head()


stats.to_sql("stats", data, if_exists = "replace", index = False)

cursor = data.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())


cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

for result in cursor.fetchall():
    print(result[0])

[('stats',)]
CREATE TABLE "stats" (
"Rk" INTEGER,
  "Player" TEXT,
  "Pos" TEXT,
  "Tm" TEXT,
  "MP" REAL,
  "FG" REAL,
  "FG_P" REAL,
  "THRP" REAL,
  "THRP_P" REAL,
  "FT" REAL,
  "FT_P" REAL,
  "TRB" REAL,
  "AST" REAL,
  "STL" REAL,
  "BLK" REAL,
  "TOV" REAL,
  "PTS" REAL,
  "Season" TEXT,
  "MVP" INTEGER
)


In [231]:
def query_stats(points, assists, blocks, steals, rebounds, points_, assists_, blocks_, steals_, rebounds_, thr_pt_pct, thr_pt, ft_pct, team1,team2, pos, mvp):
    '''
    SQL query function pulling stats.
    
    parameters: 
   
    outputs:
    table (pandas dataframe): dataframe with queried data
    '''
    
    cmd = \
    f"""SELECT S.PLAYER, S.POS, S.TM, S.FG, S.FG_P, S.THRP, S.THRP_P, S.FT, S.FT_P, S.PTS, S.AST, S.BLK, S.STL, S.TRB, S.TOV, S.SEASON, S.MVP
    FROM stats S 
    WHERE S.PTS >= {points} AND S.AST >= {assists} AND S.BLK >= {blocks} AND S.STL >= {steals} AND S.TRB >= {rebounds} AND 
          S.PTS <= {points_} AND S.AST <= {assists_} AND S.BLK <= {blocks_} AND S.STL <= {steals_} AND S.TRB <= {rebounds_} AND
          S.THRP >= {thr_pt} AND S.THRP_P >= {thr_pt_pct} AND S.FT_P >= {ft_pct}  AND S.MVP == {mvp}
    """
    
    data = sqlite3.connect("data.db")
    table = pd.read_sql_query(cmd, data)
    
    if team1 and team2 != None:
        table1 = table[table['Tm']==f"{team1}"]
        table2 = table[table['Tm']==f"{team2}"]
        frames = [table1,table2]
        table = pd.merge(table1,table2,how="inner",on="Player")
        table=table.drop_duplicates()
    elif team1 != None:
        table= table[table['Tm']==f"{team1}"]
    elif team2 != None:
        table= table[table['Tm']==f"{team2}"]
    if pos != None:
        table= table[table['Pos']==f"{pos}"]
    data.close()
    
    return table # return pandas dataframe

# defaults
points = 0
assists = 0
blocks = 0
steals = 0
rebounds = 0
points_ = 100 
assists_ = 100
blocks_ = 100 
steals_ = 100 
rebounds_ = 100 
thr_pt = 0
thr_pt_pct = 0 
ft_pct = 0 
team1 = None
team2 = None
pos = None
mvp = False

base = query_stats(points, assists, blocks, steals, rebounds, points_, assists_, blocks_, steals_, rebounds_, thr_pt_pct, thr_pt, ft_pct, team1,team2, pos, mvp)
base

,Player,Pos,Tm,FG,FG_P,THRP,THRP_P,FT,FT_P,PTS,AST,BLK,STL,TRB,TOV,Season,MVP
0,Mahmoud Abdul-Rauf,PG,SAC,3.3,0.377,0.2,0.161,0.5,1.000,7.3,1.9,0.0,0.5,1.2,0.6,1997-98,0
1,Tariq Abdul-Wahad,SG,SAC,2.4,0.403,0.1,0.211,1.4,0.672,6.4,0.9,0.2,0.6,2.0,1.1,1997-98,0
2,Shareef Abdur-Rahim,SF,MEM,8.0,0.485,0.3,0.412,6.1,0.784,22.3,2.6,0.9,1.1,7.1,3.1,1997-98,0
3,Cory Alexander,PG,SAS,1.6,0.414,0.5,0.313,0.7,0.676,4.5,1.9,0.1,0.7,1.3,1.3,1997-98,0
4,Cory Alexander,PG,DEN,4.8,0.435,2.0,0.411,2.4,0.846,14.0,6.0,0.3,2.0,4.3,2.8,1997-98,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10752,Thaddeus Young,PF,SAS,2.8,0.578,0.0,0.000,0.4,0.455,6.1,2.3,0.3,0.9,3.6,1.2,2021-22,0
10753,Thaddeus Young,PF,TOR,2.6,0.465,0.7,0.395,0.5,0.481,6.3,1.7,0.4,1.2,4.4,0.8,2021-22,0
10754,Trae Young,PG,ATL,9.4,0.460,3.1,0.382,6.6,0.904,28.4,9.7,0.1,0.9,3.7,4.0,2021-22,0
10755,Omer Yurtseven,C,MIA,2.3,0.526,0.0,0.091,0.7,0.623,5.3,0.9,0.4,0.3,5.3,0.7,2021-22,0


In [102]:

def questionr1():
        team1 = team_1
        questionr1 = f"Played for {team1}"
        globals().update(locals())
    def questionc1():
        points=points_scored
        questionc1 = f"Scored more than {points} points"
        globals().update(locals())
    def questionr2():
        pos="PG"
        globals().update(locals())
    def questionc2():
        assists=5
        globals().update(locals()) 

In [376]:
import numpy as np
import random
from random import randint

In [377]:
class choose_question():
    """
    Will randomly generate questions
    """
    def teams():
        team_choice=list(base["Tm"].unique())
        team_q1 = random.sample(team_choice,2)
        global team_1,team_2
        team_1 = team_q1[0]
        team_2 = team_q1[1]
    def pos():
        pos_choice=list(base["Pos"].unique())
        pos = random.choice(pos_choice)
        
    def points():
        global points_scored
        points_scored = randint(0,15)
    def mvp():
        mvp = random.choice([True, False])
        
list_of_question=[choose_question.points,choose_question.pos]
choose_question.teams()
#runs random team gen
loq = random.choices(list_of_question,k=2)
#chooses 2 questions and runs
global qr1,qr2
qr1=loq[0]()
qr2=loq[1]()

In [378]:
class Question:
    """
    Will ask question and also compiles list of answers
    """
    def r1c1 ():
        #Game.default()
        global team1,team2,r1c1,a,b
        team1 = team_1
        team2 = team_2
        a = query_stats(points, assists, blocks, steals, rebounds, points_, assists_, blocks_, steals_, rebounds_, thr_pt_pct, thr_pt, ft_pct, team1, team2, pos, mvp)
        b=a['Player'].unique()
        r1c1 = a
        Game.default()
        return(f"Played for {team1}", f"Played for {team2}",list(b))



In [379]:
class Answer:
    """
    Checks user input
    """
    def r1c1(player_name):
        r1c1
        return(player_match(player_name))
        

In [380]:
class Game:
    #global a
    #a = query_stats(points, assists, blocks, steals, rebounds, points_, assists_, blocks_, steals_, rebounds_, thr_pt_pct, thr_pt, ft_pct, team1, team2, pos, mvp)
    """
    Checks players
    """
    def default():
        points=0
        assists = 0
        blocks = 0
        steals = 0
        rebounds = 0
        points_ = 100 
        assists_ = 100
        blocks_ = 100 
        steals_ = 100 
        rebounds_ = 100 
        thr_pt = 0
        thr_pt_pct = 0 
        ft_pct = 0 
        team = None
        pos = None
        mvp = False
        globals().update(locals())
        #return (points, assists, blocks, steals, rebounds, points_, assists_, blocks_, steals_, rebounds_, thr_pt_pct, thr_pt, ft_pct, team, pos, mvp)
    
    def player_match(player_name):
        global b
        b=a['Player'].unique()
        result = np.any(b == player_name)
        #print(b)
        return(result)    

In [381]:
Question.r1c1()


('Played for LAL',
 'Played for ORL',
 ['Derek Harper',
  'Tyronn Lue',
  'Sean Rooks',
  'Horace Grant',
  'Brian Cook',
  'Von Wafer',
  'Maurice Evans',
  'Shammond Williams',
  'Trevor Ariza',
  'Matt Barnes',
  'Josh McRoberts',
  'Earl Clark',
  'Chris Duhon',
  'Dwight Howard',
  'Jodie Meeks',
  'Ronnie Price',
  'Brandon Bass',
  'Channing Frye',
  'Moritz Wagner'])

In [382]:
Answer.r1c1("Russell Westbrook")

['Derek Harper' 'Tyronn Lue' 'Sean Rooks' 'Horace Grant' 'Brian Cook'
 'Von Wafer' 'Maurice Evans' 'Shammond Williams' 'Trevor Ariza'
 'Matt Barnes' 'Josh McRoberts' 'Earl Clark' 'Chris Duhon' 'Dwight Howard'
 'Jodie Meeks' 'Ronnie Price' 'Brandon Bass' 'Channing Frye'
 'Moritz Wagner']


False

In [327]:
list_of_question=[choose_question.teams,choose_question.points,choose_question.pos]
qr1 = random.choices(list_of_question,k=2)
qr1[0]()



In [289]:
choose_question.teams()
choose_question.points()